# What is Abstractive Text Summarization?

**Text Summarization - as the name suggests - involves generating short summaries of text data, in a few words or sentences. A good example of this in day-to-day life is the Inshorts news summary app, which generates summaries upto max of ~ 60 words.**
* Extractive Summarization: This involves using the exact same sentences from the text in the summary, hence extractive.
* Abstractive Summarization: Here, we capture the semantics of the sentence, and generate summaries using words, which could come from anywhere in our data      vocabulary.

**The following dataset will be used for abstractive text summarization:**
[Kaggle Abstractive Text Summarization](https://www.kaggle.com/sunnysai12345/news-summary)

# I. Importing Libraries and Data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-summary/news_summary_more.csv
/kaggle/input/news-summary/news_summary.csv


In [2]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, TimeDistributed, LSTM, Embedding, Input
from keras import Model
import re
import string
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/kaggle/input/news-summary/news_summary.csv', encoding='latin-1')
data_more = pd.read_csv('/kaggle/input/news-summary/news_summary_more.csv', encoding='latin-1')
data = pd.concat([data, data_more], axis=0).reset_index(drop=True)
data

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
...,...,...,...,...,...,...
102910,NaN,NaN,CRPF jawan axed to death by Maoists in Chhatti...,NaN,A CRPF jawan was on Tuesday axed to death with...,NaN
102911,NaN,NaN,First song from Sonakshi Sinha's 'Noor' titled...,NaN,"'Uff Yeh', the first song from the Sonakshi Si...",NaN
102912,NaN,NaN,'The Matrix' film to get a reboot: Reports,NaN,"According to reports, a new version of the 199...",NaN
102913,NaN,NaN,Snoop Dogg aims gun at clown dressed as Trump ...,NaN,A new music video shows rapper Snoop Dogg aimi...,NaN


# II. Data Preprocessing

In [4]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [5]:
StopWords = set(stopwords.words('english'))
def preprocess(text):
    new_text = text.lower() #Lowercasing text.
    new_text = re.sub(r'\([^)]*\)', '', new_text) #Removing punctuations and special characters.
    new_text = re.sub('"','', new_text) #Removing double quotes.
    new_text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_text.split(" ")]) #Replacing contractions.   
    new_text = re.sub(r"'s\b","",new_text) #Eliminating apostrophe.
    new_text = re.sub("[^a-zA-Z]", " ", new_text) #Removing non-alphabetical characters
    new_text = ' '.join([word for word in new_text.split() if word not in StopWords]) #Removing stopwords.
    new_text = ' '.join([word for word in new_text.split() if len(word) >= 3]) #Removing very short words
    return new_text

#Apply above preprocessing to both text and summary separately.
text_cleaned = []
summ_cleaned = []
for text in data['text']:
    text_cleaned.append(preprocess(text))
for summary in data['headlines']:
    summ_cleaned.append(preprocess(summary))
clean_df = pd.DataFrame()
clean_df['text'] = text_cleaned
clean_df['headline'] = summ_cleaned

#Replacing empty string summaries with nan values and then dropping those datapoints.
clean_df['headline'].replace('', np.nan, inplace=True)
clean_df.dropna(axis=0, inplace=True)

#Adding START and END tokens to summaries for later use.
clean_df['headline'] = clean_df['headline'].apply(lambda x: '<START>' + ' '+ x + ' '+ '<END>')
for i in range(10):
    print('News: ', clean_df['text'][i])
    print('Headline:', clean_df['headline'][i])
    print('\n')

News:  administration union territory daman diu revoked order made compulsory women tie rakhis male colleagues occasion rakshabandhan august administration forced withdraw decision within hours issuing circular received flak employees slammed social media
Headline: <START> daman diu revokes mandatory rakshabandhan offices order <END>


News:  malaika arora slammed instagram user trolled divorcing rich man fun alimony life wearing short clothes going gym salon enjoying vacation user commented malaika responded certainly got get damn facts right spewing know nothing
Headline: <START> malaika slams user trolled divorcing rich man <END>


News:  indira gandhi institute medical sciences patna thursday made corrections marital declaration form changing virgin option unmarried earlier bihar health minister defined virgin unmarried woman consider term objectionable institute however faced strong backlash asking new recruits declare virginity form
Headline: <START> virgin corrected unmarried ig

In [6]:
#Get max length of texts and summaries.
max_len_news = max([len(text.split()) for text in clean_df['text']])
max_len_headline = max([len(text.split()) for text in clean_df['headline']])
print(max_len_news, max_len_headline)

53 14


# III. Tokenization and Data Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(clean_df['text'], clean_df['headline'], test_size=0.2, random_state=0)

#Keras tokenizer for news text.
news_tokenizer = Tokenizer()
news_tokenizer.fit_on_texts(list(X_train))
x_train_seq = news_tokenizer.texts_to_sequences(X_train)
x_test_seq = news_tokenizer.texts_to_sequences(X_test)
x_train_pad = pad_sequences(x_train_seq, maxlen=max_len_news, padding='post') #Post padding short texts with 0s.
x_test_pad = pad_sequences(x_test_seq, maxlen=max_len_news, padding='post')
#Vocab size of texts.
news_vocab = len(news_tokenizer.word_index) + 1

#Keras Tokenizer for summaries.
headline_tokenizer = Tokenizer()
headline_tokenizer.fit_on_texts(list(y_train))
y_train_seq = headline_tokenizer.texts_to_sequences(y_train)
y_test_seq = headline_tokenizer.texts_to_sequences(y_test)
y_train_pad = pad_sequences(y_train_seq, maxlen=max_len_headline, padding='post')
y_test_pad = pad_sequences(y_test_seq, maxlen=max_len_headline, padding='post')
#Vocab size of summaries.
headline_vocab = len(headline_tokenizer.word_index) + 1

 # IV. Training Model (Encoder-Decoder Architecture)

In [11]:
K.clear_session()

embedding_dim = 300 #Size of word embeddings.
latent_dim = 500 #No. of neurons in LSTM layer.

encoder_input = Input(shape=(max_len_news, ))
encoder_emb = Embedding(news_vocab, embedding_dim, trainable=True)(encoder_input) #Embedding Layer

#Three-stacked LSTM layers for encoder. Return_state returns the activation state vectors, a(t) and c(t), return_sequences return the output of the neurons y(t).
#With layers stacked one above the other, y(t) of previous layer becomes x(t) of next layer.
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.2)
y_1, a_1, c_1 = encoder_lstm1(encoder_emb)

encoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.2)
y_2, a_2, c_2 = encoder_lstm2(y_1)

encoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.2)
encoder_output, a_enc, c_enc = encoder_lstm3(y_2)

#Single LSTM layer for decoder followed by Dense softmax layer to predict the next word in summary.
decoder_input = Input(shape=(None,))
decoder_emb = Embedding(headline_vocab, embedding_dim, trainable=True)(decoder_input)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.3, recurrent_dropout=0.2)
decoder_output, decoder_fwd, decoder_back = decoder_lstm(decoder_emb, initial_state=[a_enc, c_enc]) #Final output states of encoder last layer are fed into decoder.
 
decoder_dense = TimeDistributed(Dense(headline_vocab, activation='softmax'))
decoder_output = decoder_dense(decoder_output)

model = Model([encoder_input, decoder_input], decoder_output)
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 53)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 53, 300)      20888100    input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 53, 500), (N 1602000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
_______________________________________________________________________________________

In [12]:
#Training the model with Early Stopping callback on val_loss.
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
callback = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
history=model.fit([x_train_pad,y_train_pad[:,:-1]], y_train_pad.reshape(y_train_pad.shape[0],y_train_pad.shape[1], 1)[:,1:] ,epochs=50,callbacks=[callback],batch_size=512, validation_data=([x_test_pad,y_test_pad[:,:-1]], y_test_pad.reshape(y_test_pad.shape[0],y_test_pad.shape[1], 1)[:,1:]))

Epoch 1/50
161/161 [==============================] - 189s 1s/step - loss: 4.9080 - val_loss: 4.4553
Epoch 2/50
161/161 [==============================] - 186s 1s/step - loss: 4.5144 - val_loss: 4.3700
Epoch 3/50
161/161 [==============================] - 187s 1s/step - loss: 4.3780 - val_loss: 4.2190
Epoch 4/50
161/161 [==============================] - 185s 1s/step - loss: 4.1835 - val_loss: 4.0343
Epoch 5/50
161/161 [==============================] - 184s 1s/step - loss: 3.9568 - val_loss: 3.8520
Epoch 6/50
161/161 [==============================] - 183s 1s/step - loss: 3.7414 - val_loss: 3.6844
Epoch 7/50
161/161 [==============================] - 183s 1s/step - loss: 3.5472 - val_loss: 3.5618
Epoch 8/50
161/161 [==============================] - 183s 1s/step - loss: 3.3801 - val_loss: 3.4569
Epoch 9/50
161/161 [==============================] - 183s 1s/step - loss: 3.2291 - val_loss: 3.3849
Epoch 10/50
161/161 [==============================] - 184s 1s/step - loss: 3.0925 - val_lo

# V. Inference Stage: Making Predictions!

In [13]:
#Encoder inference model with trained inputs and outputs.
encoder_model = Model(inputs=encoder_input, outputs=[encoder_output, a_enc, c_enc])

#Initialising state vectors for decoder.
decoder_initial_state_a = Input(shape=(latent_dim,))
decoder_initial_state_c = Input(shape=(latent_dim,))
decoder_hidden_state = Input(shape=(max_len_news, latent_dim))

#Decoder inference model
decoder_out, decoder_a, decoder_c = decoder_lstm(decoder_emb, initial_state=[decoder_initial_state_a, decoder_initial_state_c])
decoder_final = decoder_dense(decoder_out)
decoder_model = Model([decoder_input]+[decoder_hidden_state, decoder_initial_state_a, decoder_initial_state_c], [decoder_final]+[decoder_a, decoder_c])

In [14]:
#Function to generate output summaries.
def decoded_sequence(input_seq):
    encoder_out, encoder_a, encoder_c = encoder_model.predict(input_seq) #Collecting output from encoder inference model.
    #Initialise input to decoder neuron with START token. Thereafter output token predicted by each neuron will be used as input for the subsequent.
    #Single elt matrix used for maintaining dimensions.
    next_input = np.zeros((1,1))
    next_input[0,0] = headline_tokenizer.word_index['start']
    output_seq = ''
    #Stopping condition to terminate loop when one summary is generated.
    stop = False
    while not stop:
        #Output from decoder inference model, with output states of encoder used for initialisation.
        decoded_out, trans_state_a, trans_state_c = decoder_model.predict([next_input] + [encoder_out, encoder_a, encoder_c])
        #Get index of output token from y(t) of decoder.
        output_idx = np.argmax(decoded_out[0, -1, :])
        #If output index corresponds to END token, summary is terminated without of course adding the END token itself.
        if output_idx == headline_tokenizer.word_index['end']: 
            stop = True
        elif output_idx>0 and output_idx != headline_tokenizer.word_index['start'] :
            output_token = headline_tokenizer.index_word[output_idx] #Generate the token from index.
            output_seq = output_seq + ' ' + output_token #Append to summary
        
        #Pass the current output index as input to next neuron.
        next_input[0,0] = output_idx
        #Continously update the transient state vectors in decoder.
        encoder_a, encoder_c = trans_state_a, trans_state_c
        
    return output_seq        

In [25]:
#Print predicted summmaries and actual summaries for 10 texts. 
#We see that some of the summaries go a bit off topic, but the domain concerned, eg. politics, cricket, etc. remains largely same. 
#Further improvements can be done with Attention mechanisms, where each subsequent word generated in the summary receives different weightage
#from earlier generated words.
predicted = []
for i in range(10):
    print('News:', X_test.iloc[i])
    print('Actual Headline:', y_test.iloc[i])
    print('Predicted Headline:', decoded_sequence(x_test_pad[i].reshape(1, max_len_news)))
    predicted.append(decoded_sequence(x_test_pad[i].reshape(1, max_len_news)).split())

News: qualcomm monday announced chinese court order banning import sale apple iphone iphone models china due software patent violations court found apple violated two qualcomm software patents around resizing photographs managing applications touchscreen apple however said iphones remain sale china
Actual Headline: <START> qualcomm wins import ban apple iphones china <END>
Predicted Headline:  qualcomm bans iphone china sales china
News: congress appointed year old amit chavda new chief gujarat unit replacing year old bharatsinh solanki held post since december comes days congress president rahul gandhi said younger generations come forward take party leadership inspired gandhi words year old shantaram naik resigned goa congress chief
Actual Headline: <START> congress appoints old amit chavda new gujarat head <END>
Predicted Headline:  congress appoints first time congress leader
News: male nurse employed delhi institute liver biliary sciences booked allegedly stealing government hospi

# VI. Evaluation- BLEU score
    (BLEU - Bilingual Evaluation Understudy: commonly used for such seq2seq tasks like Machine Translation)

* BLEU score, evaluated between 0 to 1, checks how much the predicted sequence is similar to 1 or more reference sequences, in this case our y_test data.
* Naively, it is the ratio of: No. of words in machine output matching with ref / Total words in machine output. 
* A modified version involves clipping the count of matches if a word is present is more often in the machine output than in the reference. 
   Ex: 
       The cat is on the mat (Reference)
       the the the the  (Machine output)
       As per the naive evaluation, the BLEU score would be 1, since all the's appear in the reference. But we clip that count to just 2 (see why?), and get 0.5.
* Further, if the machine output is shorter than the reference, a Brevity Penalty(BP) is applied to the BLEU. 
           BP = exp(1 - r/c), where r is len(reference) and c is len(candidate output).
  In case of longer candidates, the BLEU score already penalises it adequately in the modified case.
* Here, we use the BLEU metric available in the NLTK library.

In [26]:
summaries = list(y_test)
references = []
for summ in summaries:
    ref = summ.split()
    ref.remove('<START>')
    ref.remove('<END>')
    references.append(ref)

In [27]:
refs = []
for i in range(10):
    refs.append(references[i])
    print(refs, predicted[i])
    print(sentence_bleu(refs, predicted[i]))
    refs.remove(references[i])

[['qualcomm', 'wins', 'import', 'ban', 'apple', 'iphones', 'china']] ['qualcomm', 'bans', 'iphone', 'china', 'sales', 'china']
0.6431870218238024
[['congress', 'appoints', 'old', 'amit', 'chavda', 'new', 'gujarat', 'head']] ['congress', 'appoints', 'first', 'time', 'congress', 'leader']
0.36409302398068727
[['nurse', 'sells', 'stents', 'worth', 'stolen', 'hospital']] ['delhi', 'doctor', 'files', 'fir', 'cheating', 'bank']
0
[['nurse', 'sells', 'stents', 'worth', 'stolen', 'hospital'], ['govt', 'announces', 'aadhaar', 'like', 'unique', 'businesses']] ['govt', 'provide', 'jobs', 'poor', 'rural', 'areas']
0.6389431042462724
[['nurse', 'sells', 'stents', 'worth', 'stolen', 'hospital'], ['slams', 'facebook', 'failing', 'tackle', 'human', 'trafficking']] ['indian', 'news', 'agency', 'calls', 'offensive', 'news']
0
[['nurse', 'sells', 'stents', 'worth', 'stolen', 'hospital'], ['slams', 'facebook', 'failing', 'tackle', 'human', 'trafficking'], ['everything', 'get', 'triple', 'talaq', 'bill', '